In [33]:
#from google.colab import drive
#drive.mount('/content/drive')

In [34]:
#1
import numpy as np
import pandas as pd
#関数化
def read_excel():
    #エクセル読み込み
    #df =pd.read_excel('/content/drive/MyDrive/data/iden.xlsm',sheet_name='Sheet1')
    df =pd.read_excel('C:/Users/r27150042/Documents/python/iden/iden.xlsm',sheet_name='Sheet1')
    #df
    #dfのほしい場所だけ抜き取る
    df=df.iloc[4:14,1:33]
    #df
    #エラー値を０に変換
    df=df.fillna(0)
    #df
    #希望休（◎）を2へ変換
    df=df.replace('◎',2)
    #df
    #休日数のテーブルを取得
    holiday=df.iloc[:,31:32].reset_index(drop=True)
    holiday.columns=['休日数']
    #holiday
    #基礎テーブルを取得
    kiso=df.iloc[:,0:31].reset_index(drop=True)
    kiso.columns=[i+1 for i in range(len(kiso.columns))]
    #kiso
    return kiso,holiday

In [35]:
#エクセル読み込み
kiso,holiday=read_excel()

In [36]:
kiso

,1,2,3,4,5,6,7,8,9,10,...,22,23,24,25,26,27,28,29,30,31
0,2,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,0,0
4,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,2,0,0,0,0


In [37]:
#2

def first_gene(kiso,holiday):
    #1か月の日数
    days=len(kiso.columns)
    #days

    #基礎テーブルのコピー
    kiso_copy=kiso.copy()

    for k in range(len(kiso_copy)):

        #休日を入れる変数
        h=[]
        #休日数だけループ
        while len(h)<holiday.loc[k][0]:
            n=np.random.randint(1,days+1)
            if not n in h:
                h.append(n)

        #休日を埋め込む
        for i in h:
            if kiso_copy.loc[k][i]==0:
                kiso_copy.loc[k][i]=1
    return kiso_copy

In [38]:
holiday

,休日数
0,9
1,8
2,9
3,8
4,9
5,9
6,10
7,8
8,9
9,9


In [39]:
#第1世代
kiso_copy=first_gene(kiso,holiday)
kiso_copy

C:\Users\r27150042\AppData\Local\Temp\ipykernel_11928\3643056479.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  while len(h)<holiday.loc[k][0]:


,1,2,3,4,5,6,7,8,9,10,...,22,23,24,25,26,27,28,29,30,31
0,2,0,0,2,0,0,1,0,0,0,...,1,1,1,0,0,0,0,0,0,0
1,1,0,1,0,0,0,0,1,1,0,...,0,0,0,0,1,0,0,0,0,0
2,1,0,0,1,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
3,0,1,0,1,0,0,0,0,0,0,...,1,0,0,1,2,0,0,1,0,0
4,0,1,0,1,0,0,0,0,2,0,...,1,0,1,0,0,1,0,0,1,0
5,0,0,1,0,0,1,1,0,0,0,...,1,0,1,0,0,0,1,0,1,0
6,0,0,1,0,0,2,1,0,0,1,...,0,0,0,1,0,1,0,1,1,0
7,1,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
8,0,0,0,0,1,0,0,1,1,1,...,2,0,0,0,0,1,0,0,0,0
9,1,0,0,1,0,1,1,0,1,0,...,0,0,0,1,2,2,1,0,0,0


In [40]:
#3
#休日数の修正,2分1が不足
def holiday_fix(kiso_copy,holiday):
#s==理想休日数-予定休日数
    for k in range(len(kiso_copy)):
            #1か月の日数
        days=len(kiso.columns)

        if np.count_nonzero(kiso_copy.iloc[k:k+1])!=holiday.loc[k][0]:
            s=np.count_nonzero(kiso_copy.iloc[k:k+1])-holiday.loc[k][0]
            #buf==休み増減合計
            buf=0
            #c1==0 or 1 休みが余っている時(s>0)1その他0
            #c2==0 or 1 休みが足りない時(s≦0又はc1=0でも可)1その他0
            c1=1 if s>0 else 0
            c2=1 if c1==0 else 0
            #増減したい休日日数に達するまでループ
            while buf<abs(s):
                n=np.random.randint(1,days+1)
                if kiso_copy.loc[k][n]==c1:
                    buf+=1
                    #休日を消す
                    kiso_copy.loc[k][n]=c2

    return kiso_copy

In [41]:
#4
#希望休を1へ変換
#評価方法
def evalution_function(kiso_copy):
    eva=kiso_copy.replace(2,1)
    score=0

    for k in range(len(eva)):
        #文字列として結合
        x=''.join([str(i) for i in np.array(eva.iloc[k:k+1]).flatten()])
        #5連勤以上の評価
        score+=np.sum([((2-len(i))**2)*-1 for i in x.split('1') if len(i)>=5])
        #print(score)
        #3連休以上の評価
        score+=np.sum([((1-len(i))**2)*-1 for i in x.split('0') if len(i)>=3])
        #print(score)
        #飛び石連休の評価
        score+=-10*(len(x.split('101'))-1)
        #print(score)
    #出勤数の評価
    score+=np.sum([abs(len(eva)*0.7-(len(eva)-np.sum(eva[k])))*-4 for k in eva.columns])
    return score
    #print(score)

In [42]:
#評価
score=evalution_function(kiso_copy)
score

-608.0

In [43]:
#関数化
#1エクセル読み込み
kiso,holiday=read_excel()
#2第1世代
kiso_copy=first_gene(kiso,holiday)
#3休日数の修正
kiso_copy=holiday_fix(kiso_copy,holiday)
#4評価
score=evalution_function(kiso_copy)
score

C:\Users\r27150042\AppData\Local\Temp\ipykernel_11928\3643056479.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  while len(h)<holiday.loc[k][0]:
C:\Users\r27150042\AppData\Local\Temp\ipykernel_11928\2167550465.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if np.count_nonzero(kiso_copy.iloc[k:k+1])!=holiday.loc[k][0]:
C:\Users\r27150042\AppData\Local\Temp\ipykernel_11928\2167550465.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.i

-780.0

In [44]:
#6遺伝的アルゴリズム

In [45]:
import numpy as np
import pandas as pd
from random import random
from array import array

#def read_excel():
    #Excel読み込み
#    df=pd.read_excel('iden.xlsm',sheet_name='Sheet1')
#一様交叉確率:ep=0.5,突然変異確率:sd=0.05
def crossover(ep,sd,p1,p2):
    #1か月の日数
    days=len(p1.columns)
    #1次元化:flatten
    p1=np.array(p1).flatten()
    p2=np.array(p2).flatten()
    #子の変数:遺伝子入れる箱
    ch1=[]
    ch2=[]
    #回す:zip
    for p1_,p2_ in zip(p1,p2):
        #コインの裏表
        x = True if ep >random() else False
        if x==True:
            ch1.append(p1_)
            ch2.append(p2_)
        else:
            ch1.append(p2_)
            ch2.append(p1_)
    #突然変異
    ch1,ch2=mutation(sd,np.array(ch1).flatten(),np.array(ch2).flatten())
    #pandasに変換
    ch1=pd.DataFrame(ch1.reshape(int(len(ch1)/days),days))
    ch2=pd.DataFrame(ch2.reshape(int(len(ch2)/days),days))
    #列名の変更
    ch1.columns=[i+1 for i in range(len(ch1.columns))]
    ch2.columns=[i+1 for i in range(len(ch2.columns))]
    return ch1,ch2
#突然変異
def mutation(sd,ch1,ch2):
    x = True if sd>random() else False
    if x==True:
        rand=np.random.permutation([i for i in range(len(ch1))])
        #遺伝子の10％を変異させる
        #並び換えた先頭10％
        rand=rand[:int(len(ch1)//10)]
        for i in rand:
            #1なら0,0なら1
            if ch1[i]==1:
                ch1[i]==0

            if ch1[i]==0:
                ch1[i]==1
                #ch2も同じことをする。
    x=True if sd>random() else False
    if x==True:
        rand=np.random.permutation([i for i in range(len(ch1))])
        #並び換えた先頭10％、10で割った整数時→10％
        rand=rand[:int(len(ch1)//10)]
        for i in rand:
            #1なら0,0なら1
            if ch2[i]==1:
                ch2[i]==0
            if ch2[i]==0:
                ch2[i]==1

    return ch1,ch2


In [46]:
#7全体

In [47]:
#遺伝的アルゴリズム
#エクセル読み込み
kiso,holiday=read_excel()
#親の保存
parent=[]
for i in range(100):
    #第1世代
    kiso_copy=first_gene(kiso,holiday)
    #休日数の修正
    kiso_copy=holiday_fix(kiso_copy,holiday)
    #parent.append(kiso_copy)
    #評価
    score=evalution_function(kiso_copy)
    #第1世代を格納
    parent.append([score,kiso_copy])
#上位個体数
elite_length=20
#世代数
gene_length=50

#一様交叉確立
ep=0.5
#突然変異確立
sd=0.05

#一様交叉
#ch1 = pd.DataFrame(ch1)
#ch2 = pd.DataFrame(ch2)
#ch1,ch2=crossover(ep,sd,parent[0],parent[1])
#休日数変更
#ch1=holiday_fix(ch1,holiday)
#ch2=holiday_fix(ch2,holiday)

for i in range(gene_length):
    #点数で並び換え
    parent = sorted(parent, key=lambda x: -x[0])
    #parent=sorted(np.array(parent), key=lambda x:-x[0])
    #parent=sorted(np.array(parent[0]).flatten(), key=lambda x:-x[0])
    #p1=np.array(parent[0]).flatten()
    #上位個体を選別
    parent=parent[:elite_length]
    #各世代
    print('第'+str(i+1)+'世代')

    #最高得点の更新
    #if i ==0 or top[0]<parent[0][0]:
    if i ==0:
        top=parent[0]
            #各世代の最高得点の表示
        print(top[0])
        print(np.array(top[1]))
    else:
        if top[0]>parent[0][0]:
            parent.append(top)
            #各世代の最高得点の表示
            print(top[0])
            print(np.array(top[1]))
        else:
            top=parent[0]
            #各世代の最高得点の表示
            print(parent[0][0])
            print(np.array(parent[0][1]))

    #各世代
    #print('第'+str(i+1)+'世代')
    #各世代の最高得点の表示
    #print(top[0])
    #print(np.array(top[1]))

    #子世代
    children=[]
    #遺伝子操作
    for k1,v1 in enumerate(parent):
        for k2,v2 in enumerate(parent):
            if k1<k2:
                #一様交叉
                ch1,ch2=crossover(ep,sd,v1[1],v2[1])
                #休日数変更
                #大事
                ch1=holiday_fix(ch1,holiday)
                ch2=holiday_fix(ch2,holiday)
                #評価
                score1=evalution_function(ch1)
                score2=evalution_function(ch2)
                #子孫を格納
                children.append([score1,ch1])
                children.append([score2,ch2])

    #子を親にコピー
    parent=children.copy()

#最強個体の保存
x=top[1].replace(1,'〇').replace(2,'◎').replace(0,'')
#x.to_excel('/content/drive/MyDrive/data/shift_iden42.xlsx')
x.to_excel('C:/Users/r27150042/Documents/python/iden/shift_iden42.xlsx')

C:\Users\r27150042\AppData\Local\Temp\ipykernel_11928\3643056479.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  while len(h)<holiday.loc[k][0]:
C:\Users\r27150042\AppData\Local\Temp\ipykernel_11928\2167550465.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if np.count_nonzero(kiso_copy.iloc[k:k+1])!=holiday.loc[k][0]:
C:\Users\r27150042\AppData\Local\Temp\ipykernel_11928\2167550465.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.i

第1世代
-519.0
[[2 0 0 2 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 1 0 1 0 1 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 2 2 0 0 0 0 1 0 0 1 0 1 0 0 1 0 0 0 0 1]
 [1 0 0 0 1 1 0 1 0 0 1 0 0 0 0 1 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 1 0 1 0 0 1 0 1 0 1 0 1 0 0 0 2 0 0 0 0 0 0 2 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 2 0 0 0 0 1 1 0 0 1 1 0 1 0 0 1 1 0 0 0 0 1 0]
 [0 1 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 1 1 0 0 0 1 1 0 0]
 [0 0 0 1 0 2 1 0 1 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0 1 1 0 1 0 0]
 [0 1 1 0 0 0 0 1 0 0 0 0 2 0 1 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0]
 [1 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 2 1 1 0 0 0 0 0 0 1]
 [0 0 1 0 1 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 2 2 1 0 1 0]]
第2世代
-480.0
[[2 0 0 2 0 0 1 1 0 0 0 1 1 0 0 1 1 1 0 0 1 0 0 0 1 0 0 0 0 0 1]
 [1 0 1 0 0 0 0 0 1 1 0 2 2 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [1 0 1 1 1 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 1 0 2 0 0 0 0 1 1 2 0 0 0 0 0]
 [1 0 0 0 0 0 0 1 2 0 0 0 0 0 1 0 0 1 1 1 0 0 0 0 0 1 0 1 0 0 1]
